# Рекомендация тарифов

В вашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта курса «Статистический анализ данных»). Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — вы её уже сделали.

Постройте модель с максимально большим значением *accuracy*. Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до 0.75. Проверьте *accuracy* на тестовой выборке самостоятельно.

## Откройте и изучите файл

In [1]:
import pandas as pd #импортируем библиотеки
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.dummy import DummyClassifier


In [11]:
# откроем файл с данными

try:
    df = pd.read_csv("users_behavior.csv")
except:
    df = pd.read_csv('/datasets/users_behavior.csv')

In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


3214 строк, 5 столбцов. Пропусков данных нет. 

Названия столбцов: 

сalls — количество звонков,

minutes — суммарная длительность звонков в минутах,

messages — количество sms-сообщений,

mb_used — израсходованный интернет-трафик в Мб,

is_ultra — каким тарифом пользовался в течение месяца («Ультра» — 1, «Смарт» — 0).

## Разбейте данные на выборки

In [19]:
# разбили данные на 2 выборки: 1- это обучающая и валидационная df_train_valid, 2- это тестовая 20% df_test
df_train_valid, df_test = train_test_split(df, test_size=0.2, random_state=12345) 

# разобьем обучающую (60%) и валидационную (20%): 
df_train, df_valid = train_test_split(df_train_valid, test_size=0.2, random_state=12345)


In [5]:
print(df_train.shape)
print(df_valid.shape)
print(df_test.shape)

(2056, 5)
(515, 5)
(643, 5)


Теперь у нас есть три датасета:

df_train для обучения модели

df_valid для поверки модели на переобучение

df_test для оценки качества модели

## Исследуйте модели

В качестве целевого признака выступает столбец "is_ultra". Этот признак является категориальным. Следовательно нам необходимо решить задачу классификации.

Будем использовать следующие модели машинного обучения:

- Решающее дерево
- Случайный лес
- Логистическая регрессия

У "Решающего дерева" и "Случайного леса" есть гиперпараметры, меняя которые можно подобрать наилучшую модель. У "Решающего дерева" это максимальная глубина дерева max_depth. У "Случайного леса" это количество деревьев n_estimators.

Каждую модель обучим на обучающем наборе и проверим на валидационной выборке. Таким образом определим лучшую модель.

Объявим переменные features и target (признаки и целевой признак).

In [20]:
# создадим переменные, в целевую переменную запишем столбец is_ultra, а все остальные - в признаки
# для обучающей выборки:
features_train = df_train.drop(['is_ultra'], axis=1)
target_train = df_train['is_ultra']

# для валидационной выборки:
features_valid = df_valid.drop(['is_ultra'], axis=1) 
target_valid = df_valid['is_ultra']

Исследуем модели: 
- решающее дерево
- случайный лес
- логистическая регрессия

In [21]:
# решающее дерево, исследуем гиперпараметр max_depth от 1 до 5:
best_model = None
best_result = 0
best_depth = 0
for depth in range(1, 6):
    model_tree = DecisionTreeClassifier(random_state = 12345, max_depth = depth)
    model_tree.fit(features_train, target_train) # обучим
    predictions_valid = model_tree.predict(features_valid) # найдем предсказания на валидационной выборке
    result = accuracy_score(target_valid, predictions_valid)
    if result > best_result:
        best_model = model_tree 
        best_result = accuracy_score(target_valid, predictions_valid)
        best_depth = depth

print('Accuracy наилучшей модели решающее дерево на валидационной выборке:', best_result, "Глубина дерева:", depth) 

Accuracy наилучшей модели решающее дерево на валидационной выборке: 0.7572815533980582 Глубина дерева: 5


In [22]:
# исследуем модель - случайный лес
best_model = None
best_result = 0
best_est = 0
for est in range(1, 20):
    model_forest = RandomForestClassifier(random_state=12345, n_estimators=est) # создадим модель с заданным количеством деревьев
    model_forest.fit(features_train, target_train) # обучим модель на тренировочной выборке
    result = model_forest.score(features_valid, target_valid) # посчитаем качество модели на валидационной выборке
    if result > best_result:
        best_model = model_forest 
        best_result = model_forest.score(features_valid, target_valid)
        best_est = est

print("Accuracy наилучшей модели случайный лес на валидационной выборке:", best_result, "Количество деревьев:", best_est)

Accuracy наилучшей модели случайный лес на валидационной выборке: 0.7728155339805826 Количество деревьев: 13


In [23]:
# исследуем модель - логистическая регрессия
model_lg = LogisticRegression(random_state = 12345) # инициализируем модель 
model_lg.fit(features_train, target_train) # обучим модель на тренировочной выборке
result = model_lg.score(features_valid, target_valid) # получим метрику качества модели на валидационной выборке

print("Accuracy модели логистической регрессии на валидационной выборке:", result)

Accuracy модели логистической регрессии на валидационной выборке: 0.7203883495145631


/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Исходя из всех построенных моделей: решающее дерево, случайный лес, логистическая регрессия, можно сделать вывод, что модель с максимально большим значением accuracy это Случайный лес с количеством деревьев 13. Доля правильных ответов 0.7728155339805826.  

## Проверьте модель на тестовой выборке

In [10]:
# создадим переменные для тестовой выборки
test_features = df_test.drop(['is_ultra'], axis=1)
test_target = df_test['is_ultra']

In [11]:
test_predictions = model_forest.predict(test_features)  # найдем предсказания на тестовой выборке

In [12]:
accuracy_test = model_forest.score(test_features, test_target)

In [13]:
print("Accuracy модели случайный лес на тестовой выборке:", accuracy_test)

Accuracy модели случайный лес на тестовой выборке: 0.776049766718507


## (бонус) Проверьте модели на адекватность

In [14]:
# формируем модель, которая в качестве предиктора выдает наиболее часто встречающееся значение
dummy_clf = DummyClassifier(strategy="most_frequent", random_state=12345)

In [15]:
# обучим ее на тренировочной выборке:
dummy_clf.fit(features_train, target_train)

DummyClassifier(constant=None, random_state=12345, strategy='most_frequent')

In [16]:
# сделаем предсказание:
dummy_clf.predict(features_train)

array([0, 0, 0, ..., 0, 0, 0])

In [17]:
# расчитаем долю правильных ответов:
result = dummy_clf.score(features_train, target_train)
result

0.6974708171206225

# Вывод


- Были рассмотрены три модели: "Решающее дерево", "Случайный лес" и "Логистическая регрессия". 
В каждой модели, изменяя ее гиперпараметры, мы нашли нашли лучший вариант с наибольшим количеством правильных ответов. Путем простого сравнения лучших моделей по доле правильных ответов подходит модель "Случайного леса" с  количеством деревьев 13.
- Проверив модель на тестовой выборке "Accuracy модели случайный лес на тестовой выборке: 0.776049766718507", убедились что модель работает правильно. 



## Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x] Jupyter Notebook открыт
- [x] Весь код исполняется без ошибок
- [x] Ячейки с кодом расположены в порядке исполнения
- [x] Выполнено задание 1: данные загружены и изучены
- [x] Выполнено задание 2: данные разбиты на три выборки
- [x] Выполнено задание 3: проведено исследование моделей
    - [x] Рассмотрено больше одной модели
    - [x] Рассмотрено хотя бы 3 значения гипепараметров для какой-нибудь модели
    - [x] Написаны выводы по результатам исследования
- [x] Выполнено задание 3: Проведено тестирование
- [x] Удалось достичь accuracy не меньше 0.75
